## Importación de Librerías y Descarga de recursos NLTK

In [7]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unicodedata import normalize
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.models import word2vec
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from collections import Counter
import random
import time

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\velez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\velez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Funciones de Preprocesamiento

In [8]:
def limpiador(text):
    """Remueve etiquetas HTML, caracteres no alfanuméricos y conserva emoticones."""
    text = re.sub('<[^<]*>', '', text)
    emoticons = ''.join(re.findall(r'[:;=]-+[\)\(pPD]+', text))
    text = re.sub(r'\W+', ' ', text.lower()) + emoticons.replace('-', '')
    return text

def remover_emoji(text):
    """Elimina emojis del texto."""
    emoji_pattern = re.compile("[" 
                               u"\U0001F600-\U0001F64F"  
                               u"\U0001F300-\U0001F5FF"  
                               u"\U0001F680-\U0001F6FF"  
                               u"\U0001F1E0-\U0001F1FF"  
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

## Implementación Propia de Word2Vec

In [9]:
class SkipGramWord2Vec:
    def __init__(self, text, window_size=2, embedding_dim=100, learning_rate=0.01, 
                 epochs=5, negative_samples=5):
        """
        Implementación del modelo Skip-Gram de Word2Vec con negative sampling
        
        Args:
            text: Lista de oraciones tokenizadas
            window_size: Tamaño de la ventana de contexto
            embedding_dim: Dimensión de los vectores de palabras
            learning_rate: Tasa de aprendizaje para SGD
            epochs: Número de épocas de entrenamiento
            negative_samples: Número de muestras negativas por muestra positiva
        """
        self.text = text
        self.window_size = window_size
        self.embedding_dim = embedding_dim
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.negative_samples = negative_samples
        
        # Construir vocabulario
        self.build_vocabulary()
        
        # Inicializar matrices de vectores
        self.W1 = np.random.uniform(-0.5/self.embedding_dim, 0.5/self.embedding_dim, 
                                   (self.vocab_size, self.embedding_dim))
        self.W2 = np.random.uniform(-0.5/self.embedding_dim, 0.5/self.embedding_dim,
                                   (self.vocab_size, self.embedding_dim))
        
        # Crear tabla para negative sampling
        self.create_negative_sampling_table()
        
    def build_vocabulary(self):
        """Construye el vocabulario a partir del corpus de texto"""
        # Aplanar todas las oraciones en una lista de palabras
        words = [word for sentence in self.text for word in sentence]
        
        # Contar frecuencia de cada palabra
        word_counts = Counter(words)
        self.total_words = len(words)
        
        # Crear mapeos palabra → índice y viceversa
        self.word_to_idx = {word: i for i, word in enumerate(word_counts.keys())}
        self.idx_to_word = {i: word for word, i in self.word_to_idx.items()}
        self.vocab_size = len(self.word_to_idx)
        
        # Calcular frecuencias normalizadas para negative sampling
        self.word_freqs = np.zeros(self.vocab_size)
        for word, count in word_counts.items():
            self.word_freqs[self.word_to_idx[word]] = count / self.total_words
        
        # Elevar a la potencia 3/4 como se menciona en el paper de Word2Vec
        self.word_freqs = np.power(self.word_freqs, 0.75)
        self.word_freqs = self.word_freqs / np.sum(self.word_freqs)
        
        print(f"Tamaño del vocabulario: {self.vocab_size}")
    
    def create_negative_sampling_table(self, table_size=1000000):
        """Crea una tabla para muestreo negativo eficiente"""
        self.neg_table = np.zeros(table_size, dtype=np.int32)
        
        p = 0
        i = 0
        
        # Llenar la tabla según las frecuencias de palabras
        for word_idx in range(self.vocab_size):
            p += self.word_freqs[word_idx]
            # Calcular cuántas posiciones ocupará esta palabra en la tabla
            count = int(p * table_size)
            self.neg_table[i:count] = word_idx
            i = count
            
        # Si hay posiciones restantes, llenarlas con la última palabra
        if i < table_size:
            self.neg_table[i:] = word_idx
        
        # Mezclar la tabla para mayor aleatoriedad
        np.random.shuffle(self.neg_table)
    
    def generate_training_pairs(self):
        """Genera pares de entrenamiento (palabra central, palabra de contexto)"""
        pairs = []
        
        # Para cada oración
        for sentence in self.text:
            word_indices = [self.word_to_idx[word] for word in sentence if word in self.word_to_idx]
            
            # Para cada palabra en la oración
            for center_pos in range(len(word_indices)):
                center_word_idx = word_indices[center_pos]
                
                # Tomar palabras dentro de la ventana
                context_start = max(0, center_pos - self.window_size)
                context_end = min(len(word_indices), center_pos + self.window_size + 1)
                
                for context_pos in range(context_start, context_end):
                    if context_pos != center_pos:  # No incluir la palabra central como su propio contexto
                        context_word_idx = word_indices[context_pos]
                        pairs.append((center_word_idx, context_word_idx))
        
        return pairs
    
    def sigmoid(self, x):
        """Función sigmoide"""
        clip_x = np.clip(x, -15, 15)  # Prevenir overflow numérico
        return 1 / (1 + np.exp(-clip_x))
    
    def sample_negative_words(self, positive_word_idx, n_samples):
        """Muestrea palabras negativas (distintas a la palabra positiva)"""
        negative_samples = []
        while len(negative_samples) < n_samples:
            neg_idx = self.neg_table[np.random.randint(0, len(self.neg_table))]
            if neg_idx != positive_word_idx and neg_idx not in negative_samples:
                negative_samples.append(neg_idx)
        return negative_samples
    
    def train_pair(self, center_word_idx, context_word_idx):
        """Entrena el modelo con un par palabra central - palabra contexto"""
        # 1. Obtener vectores actuales
        center_vector = self.W1[center_word_idx]
        context_vector = self.W2[context_word_idx]
        
        # 2. Forward pass para el par positivo
        dot_product = np.dot(center_vector, context_vector)
        sigmoid_val = self.sigmoid(dot_product)
        
        # 3. Calcular gradiente para el ejemplo positivo (objetivo = 1)
        gradient = self.learning_rate * (1 - sigmoid_val)
        
        # 4. Actualizar vectores del par positivo
        center_update = gradient * context_vector
        context_update = gradient * center_vector
        
        self.W1[center_word_idx] += center_update
        self.W2[context_word_idx] += context_update
        
        # 5. Muestrear palabras negativas
        negative_indices = self.sample_negative_words(context_word_idx, self.negative_samples)
        
        # 6. Forward pass y actualización para cada ejemplo negativo
        for neg_idx in negative_indices:
            neg_vector = self.W2[neg_idx]
            
            # Forward pass (objetivo = 0)
            neg_dot = np.dot(center_vector, neg_vector)
            neg_sigmoid = self.sigmoid(neg_dot)
            
            # Calcular gradiente (objetivo = 0)
            neg_gradient = self.learning_rate * (-neg_sigmoid)
            
            # Actualizar vectores
            center_neg_update = neg_gradient * neg_vector
            neg_update = neg_gradient * center_vector
            
            self.W1[center_word_idx] += center_neg_update
            self.W2[neg_idx] += neg_update
    
    def train(self):
        """Entrena el modelo de Word2Vec"""
        start_time = time.time()
        for epoch in range(self.epochs):
            # Generar pares de entrenamiento
            training_pairs = self.generate_training_pairs()
            
            # Mezclar para SGD
            random.shuffle(training_pairs)
            
            print(f"Época {epoch+1}/{self.epochs}, Pares: {len(training_pairs)}")
            
            # Entrenar cada par
            for i, (center_idx, context_idx) in enumerate(training_pairs):
                self.train_pair(center_idx, context_idx)
                
                # Mostrar progreso
                if (i+1) % 10000 == 0 or i == len(training_pairs) - 1:
                    print(f"  Progreso: {i+1}/{len(training_pairs)}")
        
        training_time = time.time() - start_time
        print(f"Entrenamiento completado en {training_time:.2f} segundos")
        self.training_time = training_time
    
    def get_word_vector(self, word):
        """Obtiene el vector de una palabra específica"""
        if word in self.word_to_idx:
            return self.W1[self.word_to_idx[word]]
        else:
            return np.zeros(self.embedding_dim)  # Vector de ceros para palabras desconocidas
    
    def get_document_vector(self, document):
        """Calcula el vector promedio para un documento (lista de palabras)"""
        vectors = [self.get_word_vector(word) for word in document if word in self.word_to_idx]
        if not vectors:
            return np.zeros(self.embedding_dim)
        return np.mean(vectors, axis=0)

## Función para evaluación de Modelos

In [10]:
def calculate_metrics(y_true, y_pred):
    conf_mat = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_mat.ravel()
    
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0  # Precisión positiva (PPV)
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Precisión negativa (NPV)
    
    return {
        "Accuracy": accuracy,
        "Sensibilidad": sensitivity,
        "Especificidad": specificity,
        "FPR": fpr,
        "FNR": fnr,
        "PPV": ppv,
        "NPV": npv
    }

## Función Para Calcular Métricas de Evaluación

In [11]:
def calculate_metrics(y_true, y_pred):
    conf_mat = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_mat.ravel()
    
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0  # Precisión positiva (PPV)
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Precisión negativa (NPV)
    
    return {
        "Accuracy": accuracy,
        "Sensibilidad": sensitivity,
        "Especificidad": specificity,
        "FPR": fpr,
        "FNR": fnr,
        "PPV": ppv,
        "NPV": npv
    }

## Función para Comparar

In [ ]:
def comparar_word2vec():
    # Cargamos el dataset
    print("Cargando dataset...")
    df = pd.read_csv('movie_data.csv')
    print(f"Dataset cargado. Dimensiones: {df.shape}")
    
    # Preprocesamiento
    print("Aplicando preprocesamiento...")
    df['review'] = df['review'].apply(limpiador)
    df['review'] = df['review'].apply(remover_emoji)
    df['review'] = df['review'].apply(lambda text: normalize("NFKD", text)
                                      .encode("ascii", "ignore")
                                      .decode("utf-8", "ignore"))
    
    # Tokenización y eliminación de stopwords para ambos modelos
    stop_words = set(stopwords.words('english'))
    tokenized = [word_tokenize(corpus) for corpus in df["review"].values]
    tokens_sin_stop = [[word for word in sublist if word.lower() not in stop_words] 
                       for sublist in tokenized]
    tokenized_final = [list(filter(lambda x: len(x) > 1, document)) 
                       for document in tokens_sin_stop]
    
    # Para trabajar con una muestra más pequeña (opcional)
    # sample_size = 1000
    # tokenized_final = tokenized_final[:sample_size]
    # y = df["sentiment"].values[:sample_size]
    
    # Etiquetas (sentimientos)
    y = df["sentiment"].values
    
    # Parámetros comunes
    feature_size = 100  # Dimensión de los vectores
    window_size = 5     # Tamaño de ventana contextual
    min_word_count = 5  # Frecuencia mínima de palabras
    epochs = 3          # Épocas de entrenamiento (reducido para la implementación propia)
    
## Modelo 1: Word2Vec de Gensim con Bosques Aleatorios
    print("\n### MODELO 1: WORD2VEC (GENSIM) + RANDOM FOREST ###")
    
    # Entrenamos el modelo Word2Vec de Gensim
    print("Entrenando Word2Vec de Gensim...")
    start_time = time.time()
    word_vec_gensim = word2vec.Word2Vec(
        tokenized_final, 
        vector_size=feature_size,
        window=window_size, 
        min_count=min_word_count,
        epochs=epochs, 
        seed=42
    )
    gensim_time = time.time() - start_time

    print(f"Entrenamiento de Word2Vec (Gensim) completado en {gensim_time:.2f} segundos")
    
    # Para cada documento, promediamos los vectores de sus palabras
    X_gensim = []
    for doc in tokenized_final:
        valid_words = [word for word in doc if word in word_vec_gensim.wv.key_to_index]
        if valid_words:
            doc_vector = np.mean([word_vec_gensim.wv[word] for word in valid_words], axis=0)
        else:
            doc_vector = np.zeros(feature_size)
        X_gensim.append(doc_vector)
    
    X_gensim = np.array(X_gensim)
    
    # Entrenar Random Forest
    print("Entrenando Random Forest...")
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_gensim, y)
    
    # Evaluación
    y_pred_rf = rf_model.predict(X_gensim)
    rf_metrics = calculate_metrics(y, y_pred_rf)
    rf_metrics["Modelo"] = "Word2Vec (Gensim) + Random Forest"
    rf_metrics["Tiempo de entrenamiento Word2Vec (s)"] = gensim_time
    
    print(f"Accuracy del modelo Random Forest: {rf_metrics['Accuracy']:.4f}")

## Modelo 2: Word2Vec Propio con MLP
    print("\n### MODELO 2: WORD2VEC (PROPIO) + MLP ###")
    
    # Entrenar Word2Vec propio
    print("Entrenando Word2Vec propio...")
    custom_w2v = SkipGramWord2Vec(
        text=tokenized_final,
        window_size=window_size,
        embedding_dim=feature_size,
        learning_rate=0.025,
        epochs=epochs,
        negative_samples=5
    )
    
    custom_w2v.train()
    
    # Generar vectores de documento promediando vectores de palabras
    X_custom = []
    for doc in tokenized_final:
        doc_vector = custom_w2v.get_document_vector(doc)
        X_custom.append(doc_vector)
    
    X_custom = np.array(X_custom)
    
    # Entrenar MLP con los vectores generados
    print("Entrenando MLP...")
    mlp_model = MLPClassifier(
        hidden_layer_sizes=(100, 50),
        activation='relu',
        solver='adam',
        alpha=0.0001,
        batch_size='auto',
        learning_rate='adaptive',
        max_iter=200,
        random_state=42
    )
    
    mlp_model.fit(X_custom, y)
    
    # Evaluación
    y_pred_mlp = mlp_model.predict(X_custom)
    mlp_metrics = calculate_metrics(y, y_pred_mlp)
    mlp_metrics["Modelo"] = "Word2Vec (Propio) + MLP"
    mlp_metrics["Tiempo de entrenamiento Word2Vec (s)"] = custom_w2v.training_time
    
    print(f"Accuracy del modelo MLP: {mlp_metrics['Accuracy']:.4f}")

## COMPARACIÓN DE RESULTADOS
        # Crear DataFrame con resultados
    results_df = pd.DataFrame([rf_metrics, mlp_metrics])
    
    print("\n### COMPARACIÓN DE RESULTADOS ###")
    print(results_df.to_string())
    
    # Visualizar comparación de accuracy
    plt.figure(figsize=(10, 6))
    models = results_df["Modelo"].values
    accuracies = results_df["Accuracy"].values
    
    plt.bar(models, accuracies, color=['skyblue', 'lightgreen'])
    plt.title('Comparación de Accuracy entre modelos')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    plt.xticks(rotation=15, ha='right')
    plt.tight_layout()
    plt.savefig('comparacion_word2vec.png')
    
    # Imprimir matriz de confusión para cada modelo
    print("\nMatriz de confusión - Random Forest:")
    print(confusion_matrix(y, y_pred_rf))
    
    print("\nMatriz de confusión - MLP:")
    print(confusion_matrix(y, y_pred_mlp))
    
    # Reporte de clasificación
    print("\nReporte de clasificación - Random Forest:")
    print(classification_report(y, y_pred_rf))
    
    print("\nReporte de clasificación - MLP:")
    print(classification_report(y, y_pred_mlp))
    
    return results_df

if __name__ == "__main__":
    comparar_word2vec()
    